In [2]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
from pypinyin import pinyin, Style

In [3]:
filename = '竹叶山流量-0418.xlsx'
raw_data = pd.read_excel(filename, sheet_name='summary')

In [4]:
# help(pd.read_excel)
raw_data.head()

,序号,方向,进口道方向,日期,早晚高峰,时间,开始时间,过车流量/辆,O,D
0,1,竹叶山东向西,东,2022年4月18,早高峰,7:00-7:15,07:00:00,324,331159364#2.526,826096563#0
1,2,竹叶山东向西,东,2022年4月18,早高峰,7:15-7:30,07:15:00,401,331159364#2.526,826096563#0
2,3,竹叶山东向西,东,2022年4月18,早高峰,7:30-7:45,07:30:00,379,331159364#2.526,826096563#0
3,4,竹叶山东向西,东,2022年4月18,早高峰,7:45-8:00,07:45:00,416,331159364#2.526,826096563#0
4,5,竹叶山东向西,东,2022年4月18,早高峰,8:00-8:15,08:00:00,379,331159364#2.526,826096563#0


In [5]:
type(raw_data.loc[0, '开始时间'])

datetime.time

In [6]:
def get_dir(in_dir):
    '''
    in_dir : 进口方向
    L_S_R_dir: 左、直、右
    '''
    if in_dir == '北':
        L_S_R_dir = ['东', '南', '西']
    elif in_dir == '东':
        L_S_R_dir = ['南', '西', '北']
    elif in_dir == '南':
        L_S_R_dir = ['西', '北', '东']
    else:
        L_S_R_dir = ['北', '东', '南']

    return L_S_R_dir


def get_flow_ID(inbound, outbound, index):
    hans = inbound+outbound
    tmp = pinyin(hans, style=Style.NORMAL, heteronym=False, errors='default', strict=True)
    res = tmp[0][0] + '_' + tmp[1][0] + '_' + str(index)
    return res


def pretty_xml(elem, level=0):
    '''
    增加换行符
    '''
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            pretty_xml(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [7]:
get_flow_ID('北','东', 3)

'bei_dong_3'

In [47]:
data = raw_data.copy()

zao_time =data.loc[0:7, '开始时间'].reset_index(drop= True)
wan_time = data.loc[8:15, '开始时间'].reset_index(drop= True)


In [52]:
flow_t

,序号,方向,进口道方向,日期,早晚高峰,时间,开始时间,过车流量/辆,O,D
0,1,竹叶山东向西,东,2022年4月18,早高峰,7:00-7:15,07:00:00,324,331159364#2.526,826096563#0
1,17,竹叶山西向东,西,2022年4月18,早高峰,7:00-7:15,07:00:00,62,331158296#1,331159373#1
2,33,竹叶山南向北,南,2022年4月18,早高峰,7:00-7:15,07:00:00,191,826105809#13,897570588
3,49,竹叶山北向南,北,2022年4月18,早高峰,7:00-7:15,07:00:00,162,897570589,27318402#1


In [60]:

root = ET.Element('routes')       # 创建节点
tree = ET.ElementTree(root)     # 创建文档

t = 0 # 从一个时间开始遍历

flow_t = data.loc[data['开始时间']==zao_time[t], :] # 时间 筛选
flow_t.index = np.arange(len(flow_t)) # reset the index

for i in range(len(flow_t)): # 四个进口道方向
    flow_t_i = flow_t.iloc[i, :]

    inbound = flow_t_i.loc['进口道方向']
    # print(inbound)
    LSR_dir = get_dir(inbound) # 左、直、右
    LSR_fraction = np.array([0.2, 0.5, 0.3]) # 左、直、右
    LSR_color = ['1,0,0','0,1,0','0,0,1']
    # print(LSR_dir)
    LSR_num = flow_t_i.loc['过车流量/辆'] * LSR_fraction
    LSR_num = np.round(LSR_num).astype('int')
    # print(LSR_num)

    for j in range(len(LSR_dir)): # 三个转向 的流量
        element = ET.Element('flow')
        
        
        outbound = LSR_dir[j]
        flow_ID = get_flow_ID(inbound, outbound, index=t) # ID
        
        b = t * (15*60) # begin
        
        e = (t+1) * (15*60) # end
        
        fr = flow_t_i['O'] # from
        
        to = flow_t.loc[flow_t['进口道方向']==outbound, 'D']
        to = to.iloc[0] # 有一个ID是数字了 # to
        # print(to)
        number = LSR_num[j] # number
        col = LSR_color[j]
        element.set('id', flow_ID)
        element.set('begin', str(b))
        element.set('end', str(e))
        element.set('from', str(fr))
        element.set('to', str(to))
        element.set('number', str(number))
        element.set('color', col)

        # element.text = ' '
        root.append(element)
        

pretty_xml(root)          # 增加换行符
tree.write('map_test.rou.xml'
           ,encoding='utf-8'
           ,xml_declaration=True
           )

In [27]:
flow_t

,序号,方向,进口道方向,日期,早晚高峰,时间,开始时间,过车流量/辆,O,D
0,1,竹叶山东向西,东,2022年4月18,早高峰,7:00-7:15,07:00:00,324,331159364#2.526,826096563#0
1,17,竹叶山西向东,西,2022年4月18,早高峰,7:00-7:15,07:00:00,62,331158296#1,331159373#1
2,33,竹叶山南向北,南,2022年4月18,早高峰,7:00-7:15,07:00:00,191,826105809#13,897570588
3,49,竹叶山北向南,北,2022年4月18,早高峰,7:00-7:15,07:00:00,162,897570589,27318402#1


In [30]:
tmp = 324 * np.array([0.2, 0.5, 0.3]) #　测试这个转向比例
np.round(tmp).astype('int')
# tmp.astype('int')


array([ 65, 162,  97])

In [28]:
# ID的设置
hans1 ='北'
hans2= '东'
hans = hans1+hans2
tmp = pinyin(hans, style=Style.NORMAL, heteronym=False, errors='default', strict=True)
tmp
print(tmp[0][0]+'_'+tmp[1][0])

bei_dong


In [9]:
key = ['北','东','南','西']
val = [0,1,2,3]
dir_dict = dict(zip(key, val))
dir_dict['北']

0

In [10]:
# 写入
# 创建节点：root = ET.Element('Root')
# 创建文档：tree = ET.ElementTree(root)
# 设置文本值：element.text = 'default'
# 设置属性：element.set('age', str(i))
# 添加节点：root.append(element)
# 写入文档：tree.write('default.xml', encoding='utf-8', xml_declaration=True)

# 但这样写入会有个问题，写入的XML会在同一行，缺少换行符，Etree本身并没有提供换行的选项，翻看了国外大神的回答，对root处理以后，再次写入将有换行符。完整代码如下：



# -*- coding:utf-8 -*-

import xml.etree.ElementTree as ET
# 增加换行符
def __indent(elem, level=0):
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            __indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

root = ET.Element('Root')       # 创建节点
tree = ET.ElementTree(root)     # 创建文档

for i in range(5):
    element = ET.Element('Name')
    element.set('age', str(i))
    element.text = 'default'
    root.append(element)

__indent(root)          # 增加换行符
tree.write('default.xml', encoding='utf-8', xml_declaration=True)

# # 文档内容
# <?xml version='1.0' encoding='utf-8'?>
# <Root>
#     <Name age="0">default</Name>
#     <Name age="1">default</Name>
#     <Name age="2">default</Name>
#     <Name age="3">default</Name>
#     <Name age="4">default</Name>
# </Root>

In [89]:
import xml.etree.ElementTree as ET

def __indent(elem, level=0):
    '''
    增加换行符
    '''
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            __indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

root = ET.Element('routes')       # 创建节点
tree = ET.ElementTree(root)     # 创建文档

for i in range(5):
    element = ET.Element('flow')
    element.set('begin', str(i))
    element.set('end', str(i+15))
    # element.set(dict(zip(['from', 'to'],[1,2])))
    # element.text = ' '
    root.append(element)

__indent(root)          # 增加换行符
tree.write('default.rou.xml', encoding='utf-8', xml_declaration=True)


In [34]:

def view_file():
    v_file = 'map.view.xml'
    root = ET.Element('viewsettings')
    tree = ET.ElementTree(root)

    element = ET.Element('scheme')
    element.set('name',"real world" )
    root.append(element)

    element = ET.Element('delay')
    element.set('value', '50')
    root.append(element)

    pretty_xml(root)
    tree.write(v_file, encoding='utf-8', xml_declaration=True)
view_file()

In [ ]:
import sumolib
import  os
import subprocess

def makeConfigFile(path, net_file_name, rou_name, guiset_name, x):
    
    sumocfg_name = "ctest_{}.sumocfg".format(x)

    path_sumocfg_name = os.path.join(path, sumocfg_name)
    path_guiset_name = os.path.join(path, guiset_name)

    if os.path.exists(path_guiset_name)== False: # 生成"osm.view.xml" 文件
        with open(path_guiset_name, 'w') as f:
            f.write(
"""
<viewsettings>
    <scheme name="real world"/>
    <delay value="0"/>
</viewsettings>
""")
    

    sumo = sumolib.checkBinary("sumo")
    
    opts = [sumo,
            "-n", net_file_name, 
            "--gui-settings-file", guiset_name,
            "-r",  rou_name,
            "-e", "600",
            "--save-configuration", path_sumocfg_name,
            "--threads", "2",
            "--no-warnings", "true",
            "--duration-log.statistics",
            "--device.rerouting.adaptation-interval", "10",
            "--device.rerouting.adaptation-steps", "18",
            "-v", "--no-step-log",  
            "--ignore-route-errors", "true",
            "--collision.action", "none",
            "--start", "true",
            "--quit-on-end", "true"
            ]

    subprocess.call(opts)

    return sumocfg_name


In [ ]:
import traci

def start_sumo(rou_file, net_file, sumocfg_file, perturbation_veh_ID, perturbation= True, use_gui= True):
    
    if use_gui:
        sb = sumolib.checkBinary('sumo-gui')
    else:
        sb = sumolib.checkBinary('sumo')

    sumo = sumolib.checkBinary("sumo")
    
    opts = [sumo,
            "-n", net_file, 
            "-r",  rou_file,
            "-e", "600",
            "--step-length", "0.5",
            "--save-configuration", sumocfg_file,
            "--threads", "2",
            "--no-warnings", "true",
            "--duration-log.statistics",
            "--device.rerouting.adaptation-interval", "10",
            "--device.rerouting.adaptation-steps", "18",
            "-v", "--no-step-log",  
            "--ignore-route-errors", "true",
            "--collision.action", "none",
            ]

    subprocess.call(opts)

    perturbation_veh_ID = [str(x) for x in perturbation_veh_ID] #转换成字符型
    traci.start([sb, "-c", sumocfg_file])

    #! 仿真运
    t = 10 # 道路上的车辆数还有，就接着仿真
    while t > 0:
        if perturbation: # 手动控制速度
            veh_id = traci.edge.getLastStepVehicleIDs('gneE0') #　返回的结果是元组，是string类型
            for veh in perturbation_veh_ID:
                if veh in veh_id:
                    
                    perturbation_speed = np.random.randint(7, 13, 1) # 随机速度的范围
                    traci.vehicle.setSpeed(veh, perturbation_speed)
                    traci.vehicle.setColor(veh, (255,0,0))

        traci.simulationStep()  # 仿真跑一步
        t = traci.edge.getLastStepVehicleNumber('gneE0')

    traci.close()

In [35]:
'map_zao.rou.xml'.split(sep='.')[0]

'map_zao'

In [36]:
help(str.split)

Help on method_descriptor:

split(self, /, sep=None, maxsplit=-1)
    Return a list of the words in the string, using sep as the delimiter string.
    
    sep
      The delimiter according which to split the string.
      None (the default value) means split according to any whitespace,
      and discard empty strings from the result.
    maxsplit
      Maximum number of splits to do.
      -1 (the default value) means no limit.



In [38]:
import traci
help(traci.start)

Help on function start in module traci.main:

start(cmd, port=None, numRetries=10, label='default', verbose=False, traceFile=None, traceGetters=True, stdout=None)
    Start a sumo server using cmd, establish a connection to it and
    store it under the given label. This method is not thread-safe.

